In [ ]:
def run_lgb(data, features, cat_features):
    
    # reset_index
    data.reset_index(inplace = True, drop = True)
    
    # going to evaluate with the last 28 days
    x_train = data[data['date'] <= '2016-04-24']
    y_train = x_train['demand']
    test = data[data['date'] >= '2016-04-25']
    
    oof = np.zeros(len(x_train))
    preds = np.zeros(len(test))
    
    # GroupKFold by week, month to avoid leakage and overfitting (not entirely sure xD)
    kf = GroupKFold(5)
    # get subgroups for each week, year pair
    group = x_train['week'].astype(str) + '_' + x_train['year'].astype(str)
    for fold, (trn_idx, val_idx) in enumerate(kf.split(x_train, y_train, group)):
        print(f'Training fold {fold + 1}')
        train_set = lgb.Dataset(x_train.iloc[trn_idx][features], y_train.iloc[trn_idx], 
                                categorical_feature = cat_features)
        val_set = lgb.Dataset(x_train.iloc[val_idx][features], y_train.iloc[val_idx], 
                              categorical_feature = cat_features)
        
        # train with our custom loss function and evaluation metric
        model = lgb.train(params, train_set, num_boost_round = 10000, early_stopping_rounds = 50, 
                          valid_sets = [train_set, val_set], verbose_eval = 50, fobj = custom_asymmetric_train, 
                          feval = custom_asymmetric_valid)
    
        # predict oof
        oof[val_idx] = model.predict(x_train.iloc[val_idx][features])

        # predict test
        preds += model.predict(test[features]) / 5
        
        print('-'*50)
        print('\n')
        
    oof_rmse = np.sqrt(metrics.mean_squared_error(y_train, oof))
    print(f'Our out of folds rmse is {oof_rmse}')
        
    test = test[['id', 'date', 'demand']]
    test['demand'] = preds
    return test

The same group will not appear in two different folds (the number of distinct groups has to be at least equal to the number of folds).